## 第5週の4日目

Autogen Core - （実験的）分散

これについては（プロモーション/プレビューの意味で）ティーザーのみ公開します!!

それがあなたにとってどれほど関連性があるかわからないからです。

これのためにもっとコンテンツを追加してほしいなら、私に知らせてください。

In [1]:
# import
from dotenv import load_dotenv
from dataclasses import dataclass

from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime
from autogen_ext.tools.langchain import LangChainToolAdapter

from langchain.agents import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper

from IPython.display import display, Markdown

# 初期化
load_dotenv(override=True)
ALL_IN_ONE_WORKER = False

### メッセージクラスから始め

In [2]:
@dataclass
class Message:
    content: str

### ツールを再導入して

In [3]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

### そして、いくつかのエージェント

In [4]:
# 新しい AI エージェント プロジェクトで AutoGen を使用するかどうかの決定を支援するために、AutoGen を選択する理由 (AutoGen の利点) を調査し、簡単に回答してください。
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

# 新しい AI エージェント プロジェクトで AutoGen を使用するかどうかの決定を支援するために、AutoGen を選択しない理由と Autogen の欠点を調査し、簡単に回答してください。
instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

# プロジェクトで AutoGen を使用するかどうかを決定する必要があります。
# 研究チームは、以下の賛成理由と反対理由を挙げています。
# チームの調査結果のみに基づいて、決定理由と簡単な根拠をお知らせください。
judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [5]:
# Player1Agent、Player2Agentは全く同じ
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

# Player1Agent、Player2Agentは全く同じ
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

# PlayerAgentとの違いは、ツールを持たない点と階層構造
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:

        # ※ インスタンスを生成するのではなく取得する。
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")

        # メッセージを２つのプレイヤーに送る
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)

        # 結果
        # f"## AutoGen の利点:\n{response1.content}\n\n## AutoGen の欠点:\n{response2.content}\n\n"
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        
        # 判定
        # f"{judge}\n{result}あなたの決定と簡単な説明を返信してください"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")

        # 結果判定結果
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

### そして今 - 分散したランタイムのホスト

In [6]:
# GrpcWorkerAgentRuntimeには、Hostが必要らしい。
host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()

# SingleThreadedAgentRuntime -> GrpcWorkerAgentRuntime
# コレは、gRPC で通信するエージェントワーカーを表すクラス
# ランタイム ＝ ワーカー、ワーカープロセス的な意味合い。

if ALL_IN_ONE_WORKER: # 全てのエージェントを１つのランタイムに入れる。

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else: # エージェントごとに別々のランタイム実装を立てる

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")

response = await worker.send_message(Message(content="Go!"), agent_id)
display(Markdown(response.content))

## Pros of AutoGen:
Here are several reasons in favor of using AutoGen for your AI Agent project:

1. **Ease of Use**: AutoGen provides a user-friendly framework that simplifies the creation of multi-agent applications. This can significantly reduce the learning curve for developers.

2. **Customizability**: The platform allows developers to easily customize agents for specific tasks, enabling flexibility in tailoring functionalities to meet project requirements.

3. **Collaboration**: AutoGen facilitates cooperation between agents through natural-language conversations, which can enhance creativity and improve problem-solving capabilities.

4. **Scalability**: The framework supports the development of scalable multi-agent systems, making it suitable for projects that may expand in complexity over time.

5. **Efficiency**: Users have reported substantial time savings, with up to a 75% reduction in specific task completion times, contributing to overall project efficiency.

6. **Reduced Coordination Complexity**: AutoGen eliminates the need for custom inter-agent protocols by using natural-language handoffs, streamlining communication between agents.

7. **Supports Autonomous and Human-in-the-Loop Operations**: The framework accommodates both fully autonomous agents and those that involve human oversight, providing versatility in operational control.

These advantages make AutoGen a compelling choice for developing sophisticated AI agents in various applications.

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen in your AI agent project:

1. **Steep Learning Curve**: The graph-based approach and the requirement to understand Directed Acyclic Graphs (DAGs) and state management can make it challenging for developers, particularly those who are not tech-savvy.

2. **Complicated Documentation**: Users often find the documentation hard to read, with insufficient examples, making it difficult to effectively utilize AutoGen's capabilities.

3. **Performance with Less Powerful Models**: AutoGen performs optimally with the latest large language models, which may lead to subpar results if integrated with less powerful models.

4. **Cost Concerns**: The usage-based pricing model can become expensive quickly, especially if workflows are not optimized. There is no pay-as-you-go option, necessitating a more costly upgrade even if only minimal use is required.

5. **Limited Creativity**: AutoGen is more effective at generating content based on predefined templates and inputs, which may restrict its flexibility and creativity compared to other frameworks that allow more dynamic generation.

These factors should be carefully considered when deciding whether to implement AutoGen in your project. 

TERMINATE



## Decision:

Based on the research presented by the team, I recommend using AutoGen for the project. 

Rationale: The significant advantages, including ease of use, customizability, collaboration, and scalability, outweigh the concerns about the learning curve and documentation. AutoGen offers substantial time savings and efficient communication between agents, which will enhance overall project execution. While cost and performance with lesser models are valid concerns, the framework's strengths in facilitating sophisticated AI agent capabilities make it a compelling choice for our needs. 

TERMINATE

In [7]:
# 停止
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

await host.stop()